# **Demo:** DL Model Predicting in Real-Time

**Note:** *Don't forget to mount Google Drive.*

In [76]:
# Downloads
! pip install pymssql
! pip install colorama

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [77]:
# Imports
import pandas as pd
import pymssql
import tensorflow as tf
import time
import datetime
from google.colab import drive
import numpy as np
from colorama import Fore

In [78]:
# Read in model
model = tf.keras.models.load_model('/content/drive/MyDrive/optimum_model')

# Get credentials and connection
credentials = open('/content/drive/MyDrive/config.txt', 'r')
credentials = credentials.readlines()
username = credentials[0].strip('\n')
password = credentials[1]
database = 'group5database'
server = "gen10-data-fundamentals-22-05-sql-server.database.windows.net"
table = 'dbo.CGM_Stream'
conn = pymssql.connect(server, username, password, database)
cursor = conn.cursor()

In [81]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 20, 25)            2700      
                                                                 
 dense_3 (Dense)             (None, 20, 10)            260       
                                                                 
 dropout_2 (Dropout)         (None, 20, 10)            0         
                                                                 
 gru_1 (GRU)                 (None, 20, 10)            660       
                                                                 
 dense_4 (Dense)             (None, 20, 5)             55        
                                                                 
 dropout_3 (Dropout)         (None, 20, 5)             0         
                                                                 
 flatten_1 (Flatten)         (None, 100)              

In [82]:
# Create a recurrent loop that update reading every 10 sec
for i in range(1000):

  # Read in data
  query = f'SELECT TOP(20) Glucose FROM {table} WHERE PtID = 1 ORDER BY DeviceDtTM DESC'
  df = pd.read_sql(query,conn)
  df = df.rename(columns={'Glucose': 'glucose_lvl'})

  # Check to see if it has enough entries
  if (df.shape[0] >= 20):

    # Make prediction
    window = 20
    X = [df[0:window]]
    X = np.array(X)
    prediction = model.predict(X)

    # Display results
    print(Fore.WHITE + f'STATUS UPDATE: {datetime.datetime.today()}')
    print(f'  Predicted blood glucose level in 30 min: {prediction[0][0]:.1f} mmol/L')
    print('  Status level: ', end='')
    if (prediction >= 300):
      print(Fore.RED + 'ALARMING')
    elif (prediction >= 180):
      print(Fore.YELLOW + 'AT RISK')
    else:
      print(Fore.GREEN + 'NORMAL')
  
  # Print that the service is not ready to run
  else:
    print(Fore.WHITE + f'STATUS UPDATE: {datetime.datetime.today()}')
    print('  At least 20 blood glucose readings are required for this service!')
  print()

  # Sleep 10 sec for next reading
  time.sleep(10)

STATUS UPDATE: 2022-08-16 16:44:34.049365
  Predicted blood glucose level in 30 min: 117.5 mmol/L
  Status level: NORMAL

STATUS UPDATE: 2022-08-16 16:44:44.158378
  Predicted blood glucose level in 30 min: 126.0 mmol/L
  Status level: NORMAL

STATUS UPDATE: 2022-08-16 16:44:54.262661
  Predicted blood glucose level in 30 min: 134.0 mmol/L
  Status level: NORMAL

STATUS UPDATE: 2022-08-16 16:45:04.364382
  Predicted blood glucose level in 30 min: 136.3 mmol/L
  Status level: NORMAL

STATUS UPDATE: 2022-08-16 16:45:14.478141
  Predicted blood glucose level in 30 min: 133.3 mmol/L
  Status level: NORMAL

STATUS UPDATE: 2022-08-16 16:45:24.588912
  Predicted blood glucose level in 30 min: 140.6 mmol/L
  Status level: NORMAL

STATUS UPDATE: 2022-08-16 16:45:34.684403
  Predicted blood glucose level in 30 min: 126.6 mmol/L
  Status level: NORMAL

STATUS UPDATE: 2022-08-16 16:45:44.804797
  Predicted blood glucose level in 30 min: 126.6 mmol/L
  Status level: NORMAL

STATUS UPDATE: 2022-08-1

KeyboardInterrupt: ignored